# Floating Coupon Bond

The goal of the project is to compute the market value of a portfolio of floating coupon bonds taking into account the issuer credit risk. The input parameters to derive this curve are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* A set of forward libor rates and their fixing dates
* A set of survival probabilities and the recovery rate of the issuer
* The static data of the pool of bonds to price (nominal, start date, end date, current coupon, margin, payment frequency)

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

# BOOTSTRAP THE EONIA CURVE FROM THE OIS MARKET QUOTES 

In [1]:
from finmarkets import OvernightIndexSwap, generate_swap_dates
from project_input import today, ois_quotes

ois_pillar_dates = [today]

swaps = []

for quote in ois_quotes:
    
    Swap = OvernightIndexSwap( 
                1e6, 
                generate_swap_dates(today,quote['maturity']),
                quote['rate']
            )
    
    swaps.append(Swap)
    ois_pillar_dates.append(Swap.payment_dates[-1])
    
ois_pillar_dates = sorted(ois_pillar_dates)
n_df_vector = len(ois_pillar_dates)

In [2]:
ois_pillar_dates

[datetime.date(2013, 10, 31),
 datetime.date(2013, 11, 30),
 datetime.date(2013, 12, 31),
 datetime.date(2014, 1, 31),
 datetime.date(2014, 2, 28),
 datetime.date(2014, 3, 31),
 datetime.date(2014, 4, 30),
 datetime.date(2014, 5, 31),
 datetime.date(2014, 6, 30),
 datetime.date(2014, 7, 31),
 datetime.date(2014, 8, 31),
 datetime.date(2014, 9, 30),
 datetime.date(2014, 10, 31),
 datetime.date(2015, 1, 31),
 datetime.date(2015, 4, 30),
 datetime.date(2015, 7, 31),
 datetime.date(2015, 10, 31),
 datetime.date(2016, 10, 31),
 datetime.date(2017, 10, 31),
 datetime.date(2018, 10, 31),
 datetime.date(2019, 10, 31),
 datetime.date(2020, 10, 31),
 datetime.date(2021, 10, 31),
 datetime.date(2022, 10, 31),
 datetime.date(2023, 10, 31),
 datetime.date(2024, 10, 31),
 datetime.date(2025, 10, 31),
 datetime.date(2028, 10, 31),
 datetime.date(2033, 10, 31),
 datetime.date(2038, 10, 31),
 datetime.date(2043, 10, 31)]

In [3]:
from finmarkets import DiscountCurve

def objective_function(x): 
    
    curve = DiscountCurve(
            today, 
            ois_pillar_dates, 
            x
            )
    
    sum_sq = 0.0 
    
    for swap in swaps: 
        sum_sq += swap.npv(curve)**2
        
    return sum_sq

In [4]:
from scipy.optimize import minimize 

x0 = [1.0 for i in range(n_df_vector)]

bounds = [(0.01,100.0) for i in range(n_df_vector)]

bounds[0]= (1.0,1.0)

result = minimize(objective_function,x0,bounds = bounds)

In [5]:
result

      fun: 0.0006561289683078707
 hess_inv: <31x31 LbfgsInvHessProduct with dtype=float64>
      jac: array([  5.79535029e+05,  -2.83218913e-04,  -3.24325918e-04,
        -3.24159558e-04,  -3.11566549e-04,  -3.47202280e-04,
        -3.37286536e-04,  -2.87379257e-04,  -3.00610990e-04,
        -1.85489406e-04,  -1.04252327e-04,  -1.83492718e-04,
        -3.18568119e-03,   2.00303036e-03,   2.00801986e-03,
         1.81424140e-03,  -9.36728126e-05,   1.77063934e-03,
         2.52740309e-03,   3.84087250e-04,  -2.21504221e-03,
        -2.97836556e-03,  -2.30686074e-03,  -8.51359239e-04,
         4.26810583e-04,   1.69550760e-03,   1.45341373e-03,
        -9.38931940e-04,   1.41407593e-03,   3.52212085e-04,
        -1.37813923e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 448
      nit: 11
   status: 0
  success: True
        x: array([ 1.        ,  0.99991167,  0.99980687,  0.99970619,  0.99961015,
        0.99950111,  0.999392  ,  0.99928207,  0.99916713, 

In [6]:
objective_function(result.x)

0.0006561289683078707

In [7]:
from finmarkets import DiscountCurve
from datetime import date 

eonia_curve = DiscountCurve(
            today, 
            ois_pillar_dates, 
            result.x
            )

eonia_curve.df(date(2030,10,31))

0.6710110388246098

# INTERPOLATED FORWARD LIBOR RATE

In [8]:
from finmarkets import ForwardRateCurve
from project_input import euribor_6m
from datetime import date

fwd_pillar_rates= []
fwd_pillar_dates = []

for rates in euribor_6m: 
     fwd_pillar_rates.append(rates['rate'])

for dates in euribor_6m: 
    fwd_pillar_dates.append(dates['date'])
                
forward_rate_curve= ForwardRateCurve(
                    fwd_pillar_dates, 
                    fwd_pillar_rates
                    )               
                
forward_rate_curve.forward_rate(date(2025,7,30))

0.023797814207650272

# INTERPOLATED NDPS

In [9]:
from project_input import survival_probabilities,today
from finmarkets import CreditCurve
from datetime import date

pillar_ndps = [1.0]
ndps_pillar_dates = [today]

for ndps in survival_probabilities:
    
    pillar_ndps.append(ndps['ndp'])
    
for dates in survival_probabilities:
    
    ndps_pillar_dates.append(dates['date'])
    
credit_curve = CreditCurve(
                ndps_pillar_dates,
                pillar_ndps
                )

credit_curve.ndp(date(2025,10,31))

0.691942752218531

# CALCULATE THE MTM

In [10]:
from datetime import date 
from dateutil.relativedelta import relativedelta 

class Bond_Tools(object): 
    
    def __init__(self,pricing_date,nominal,start_date,end_date,current_coupon,margin,recovery,tenor_months=6):
        
        self.pricing_date = pricing_date 
        self.nominal = nominal
        self.start_date = start_date 
        self.end_date = end_date 
        self.current_coupon = current_coupon
        self.margin = margin 
        self.recovery = recovery 
        self.tenor_months = tenor_months 
        

    def Future_Payment_dates(self):

        dates = []
        future_dates = []

        n_months = ((self.end_date.year - self.start_date.year) * 12) + (self.end_date.month - self.start_date.month)

        for n in range(0, n_months, self.tenor_months):
            dates.append(self.start_date + relativedelta(months=n))
        dates.append(self.end_date)

        for i in range(len(dates)):

            if dates[i] > self.pricing_date: 
                future_dates.append(dates[i])

        return future_dates
        
    def Mark_To_Market(self,discounting_curve,forward_curve,credit_curve):
        
        Bond_dates = self.Future_Payment_dates()
        npv_coupons_if_no_default = 0
        recovery_if_early_default = 0 
    
        first_coupon = (((Bond_dates[0]-today).days/360)*
                        discounting_curve.df(Bond_dates[0]) *
                        self.current_coupon * credit_curve.ndp(Bond_dates[0]))

        for i in range(1,len(Bond_dates)-1):

            npv_coupons_if_no_default += (((Bond_dates[i]-Bond_dates[i-1]).days/360)*
                                         discounting_curve.df(Bond_dates[i]) *
                                         (forward_curve.forward_rate(Bond_dates[i-1])+self.margin)
                                         *credit_curve.ndp(Bond_dates[i]))

        npv_coupons_if_no_default += first_coupon
        
        npv_coupons_if_no_default = npv_coupons_if_no_default * self.nominal 
        
        
        notional_reimbursement_if_no_default = self.nominal*(discounting_curve.df(Bond_dates[-1])*
                                                credit_curve.ndp(Bond_dates[-1]))
        
        d= self.pricing_date
    
        while d <= Bond_dates[-1]:

            recovery_if_early_default += ((discounting_curve.df(d)) *
                                         (credit_curve.ndp(d) - 
                                          credit_curve.ndp(d+relativedelta(days=1))))
            d += relativedelta(days=1)  

        recovery_if_early_default = recovery_if_early_default * self.recovery * self.nominal
        
        MTM =  npv_coupons_if_no_default + notional_reimbursement_if_no_default + recovery_if_early_default
        
        return MTM

In [11]:
from project_input import bonds_to_price

nominals= []
margins = []
recoveries = []
current_coupons = []
end_dates = []
start_dates = []

for x in bonds_to_price:
       
    nominals.append(x['nominal'])

    margins.append(x['margin'])

    recoveries.append(x['recovery']) 

    current_coupons.append(x['current_coupon']) 

    end_dates.append(x['end_date'])

    start_dates.append(x['start_date'])

In [12]:
from project_input import today

Bonds = []
MTM_PORTFOLIO = 0 

for i in range(len(nominals)):
    
    Bond = Bond_Tools(today,nominals[i],start_dates[i],end_dates[i],current_coupons[i],margins[i],recoveries[i])
    Bonds.append(Bond)
    
    MTM_PORTFOLIO += (Bonds[i].Mark_To_Market(eonia_curve,forward_rate_curve,credit_curve))

MTM_PORTFOLIO

75008567.79145329

In [13]:
print("The portfolio is composed by %s Bonds and it has a MTM of: %.2f EUR" %(len(nominals),MTM_PORTFOLIO))

The portfolio is composed by 15 Bonds and it has a MTM of: 75008567.79 EUR
